In [2]:
# ======== a runnable full training procedure =========
# resnet18 + adam ,test acc ~ 99.1%
import sys
sys.path.append(r'/kaggle/input/dezero')
from imp import reload
import os, dezero

from dezero import Variable
import dezero.functions as F
reload(F)
import numpy as np

from dezero import optimizers
reload(optimizers)
from dezero import DataLoader
# from dezero.models import MLP
from dezero import models
from dezero.models import ResNet18, MLP, C5L3
from dezero.datasets import Dataset

# we can use torch's data loader
import torch
import torchvision.datasets as tv_datasets
import torchvision.transforms as tv_transforms

C:\Users\xuguy\AppData\Local\Temp\ipykernel_30000\3572444826.py:5: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload


No module named 'cupy'


In [3]:
# load data
kaggle = "/kaggle/input/cifar-10-dlhw1-2/data"
local = "../data"

# switch between test and kaggle mode
run_on_local = 1

if run_on_local:
    data_path = local
    sys.path.append('./code')
    
else:
    data_path = kaggle
    sys.path.append('/kaggle/input')

# define data transformation method：


In [4]:
# ======= prepare data ========
data_path = local
batch_size = 64
num_workers = 0
dataset1 = 'CIFAR10'

# load with pickle
# we download the mnist dataset and store it in .pkl
import pickle

# change the following lines to switch local/kaggle
# filepath = r'./dezero/MNISTdataset/mnist.pkl'
filepath = r'/kaggle/input/dezero/dezero/MNISTdataset/mnist.pkl'
with open(filepath, 'rb') as f:
    data = pickle.load(f)

# setup hyper-params


print_every = 1
hidden_size = 1000

max_epoch = 15
batch_size = 64

lr = 0.001

# we only use tv_transform to transform the ndarray data
g = tv_transforms.Compose([
                tv_transforms.RandomRotation(15),  
                tv_transforms.RandomHorizontalFlip(),
                tv_transforms.RandomAffine(0, translate=(0.1, 0.1)),  
                # tv_transforms.RandomResizedCrop(28, scale=(0.9, 1.1)), 
                # tv_transforms.ToTensor(),  
                tv_transforms.Normalize((0.1307,), (0.3081,))])  

def f(x):
    return g(torch.tensor(x, dtype=torch.float32))

# def f(x):
#     # x = x.flatten()
#     x = x.astype(np.float32)
#     x /= 255.0
#     return x

test_set = Dataset()
test_set.transform = f
test_set.data= data['test_img'].reshape(-1, 1, 28, 28)
test_set.label = data['test_label']
print(f'{test_set.data.shape}, {test_set.label.shape}')


train_set = Dataset()
train_set.transform = f
train_set.data= data['train_img'].reshape(-1, 1, 28, 28)
train_set.label = data['train_label']
print(f'{train_set.data.shape}, {train_set.label.shape}')

train_loader = DataLoader(train_set, batch_size)
test_loader = DataLoader(test_set, batch_size, shuffle = False)


(10000, 1, 28, 28), (10000,)
(60000, 1, 28, 28), (60000,)


C:\Users\xuguy\AppData\Local\Temp\ipykernel_30000\2927851848.py:15: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  data = pickle.load(f)


In [ ]:
# train

N = len(train_set)
# model = MLP((1000,10))
# model = ResNet18(pretrained=False)
model = C5L3()
# model = MLP((1000, 10), activation= F.leaky_relu)
# Adam works
optimizer = optimizers.Adam().setup(model)
if dezero.cuda.gpu_enable:
    train_loader.to_gpu()
    test_loader.to_gpu()
    model.to_gpu()


# try to load weights (check if path exists)
# if os.path.exists('my_mlp.npz'):
#     model.load_weights('my_mlp.npz')

for epoch in range(max_epoch):
    sum_loss, sum_acc = 0, 0

    for x, t in train_loader:
        y = model(x)
        loss = F.softmax_cross_entropy(y, t)
        model.cleargrads()
        loss.backward()
        acc = F.accuracy(y, t)
        optimizer.update()
        sum_loss += float(loss.data)*len(t)
        sum_acc += float(acc.data) * len(t)


    # print training info
    avg_loss = sum_loss / len(train_set)
    avg_acc = sum_acc / len(train_set)
    # if epoch % print_every == print_every-1 :
    print(f'epoch: {epoch+1}, train_avg_loss: {avg_loss:.4f}, train_avg_acc: {avg_acc:.4f}')

    sum_loss, sum_acc = 0, 0
    # print eval info
    with dezero.no_grad():
        for x, t in test_loader:
            y = model(x)
            # loss = F.softmax_cross_entropy(y, t)
            acc = F.accuracy(y, t)
            # sum_loss += float(loss.data) * len(t)
            sum_acc += float(acc.data) * len(t)

    
    # print eval(test) info
    # avg_loss = sum_loss / len(test_set)
    avg_acc = sum_acc / len(test_set)
    # if epoch % print_every == print_every-1 :
    # print(f'test_avg_loss: {avg_loss:.4f}, test_avg_acc: {avg_acc:.4f}')
    print(f'test_avg_acc: {avg_acc:.4f}')

In [20]:
# # part 2, try to repeat C5L3 using dezero framework.abs
# reload(dezero.models)
# from dezero.models import C5L3
# model = C5L3()
# x = train_set.data[1].reshape(1,1,28,28)
# model(x)


variable([[ 1.77515229 -0.7487996   0.75166254 -0.04596518  0.34729091 -1.05023357
            1.4510835   1.56595645 -2.77450248 -0.7332514 ]])

In [ ]:
# # visualize image after transform
# import matplotlib.pyplot as plt
# import numpy as np

# # remove batch dim to get an array of shape (28, 28)
# img_arr = data['train_img'][1].reshape(-1,28,28)

# img_arr = f(torch.tensor(img_arr, dtype = torch.float32))
# image = image_array.squeeze()

# # 展示图像
# plt.imshow(img_arr.squeeze(), cmap='gray')  # gray projection
# plt.axis('off')  # turn off axis
# plt.show()